We need metrics designed specifically for continous values...

Mean Absolute Error: Mean of the absolute value of errors... Can be viewed as average error. "On average you're about $100,000 off from the actual home price."

Mean Squared Error: Mean of squared errors. Takes into account when you are off by a very large amount, making MSE more popular than MAE.

Root Mean Square Error: Root of the mean of the squared errors to put the error metric in terms of y.

R Squared Values: Coefficient of determination. It's a sense of how much variance your model accounts for. 
When comparing models you should use adjusted r squared.

In [1]:
# Boiler Plate
import findspark
import numpy as np
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In this example we are going to try and predict a customer's total amount expenditure. We'll also convert realistic data into a format accepted by Spark's MLlib

In [2]:
from pyspark.ml.regression import LinearRegression

In [3]:
data = spark.read.csv('Ecommerce_Customers.csv',inferSchema=True,header=True)

In [4]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [6]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [8]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [10]:
# In this example we're just going to use the numerical features
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [13]:
assembler = VectorAssembler(inputCols=['Avg Session Length','Time on App','Time on Website','Length of Membership']
                            ,outputCol='features')

In [14]:
output = assembler.transform(data)

In [16]:
output.head(1)

Exception ignored in: <object repr() failed>
Traceback (most recent call last):
  File "/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/pyspark/ml/wrapper.py", line 76, in __del__
    SparkContext._active_spark_context._gateway.detach(self._java_obj)
AttributeError: 'VectorAssembler' object has no attribute '_java_obj'
Exception ignored in: <object repr() failed>
Traceback (most recent call last):
  File "/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/pyspark/ml/wrapper.py", line 76, in __del__
    SparkContext._active_spark_context._gateway.detach(self._java_obj)
AttributeError: 'VectorAssembler' object has no attribute '_java_obj'


[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [17]:
final_data = output.select('features','Yearly Amount Spent')

In [18]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [19]:
# 70-30 is a pretty common split
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [20]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                354|
|   mean|  496.8640504908049|
| stddev|  80.78832934655019|
|    min| 256.67058229005585|
|    max|  744.2218671047146|
+-------+-------------------+



In [21]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                146|
|   mean| 505.25441955856536|
| stddev|   75.5625022320736|
|    min| 302.18954780965197|
|    max|  765.5184619388373|
+-------+-------------------+



In [22]:
# Don't need to include our features data becauase the default is called features, which is what we have.
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [23]:
lr_model = lr.fit(train_data)

In [24]:
test_results = lr_model.evaluate(test_data)

In [25]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|  0.2665865080695653|
|   7.066576702219436|
| -3.1923853257987957|
|   4.197659088714204|
|  0.6761909239414763|
|  -4.069945082798313|
|   2.775505084531858|
|  -7.595435220915192|
| -14.339758818032351|
|-0.43632215769287086|
|  -5.871608595703663|
| -11.042257020576926|
| -1.5682439399974442|
| -17.871052304740715|
|  3.6902854745907803|
|   1.654546204195185|
|   5.247204386393946|
|   4.636632826130608|
| -11.220605353106578|
|  6.0423066543897335|
+--------------------+
only showing top 20 rows



In [26]:
test_results.rootMeanSquaredError

9.694687551807181

In [28]:
# We're explaining a lot of the error, 98%.
test_results.r2

0.9834255233111082

In [29]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



We can see that our model is pretty good based off the RMSE. If the RMSE is 10 and the actual data's mean is 500 with a standard deviation of 80 we are in pretty good shape.

Now, how can we apply this model when we only have customer features?

In [30]:
unlabeled_data = test_data.select('features')

In [31]:
# We'll simulate a situation where we only have feature data.
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.5743636841713...|
|[30.9716756438877...|
|[31.2681042107507...|
|[31.3584771924370...|
|[31.3895854806643...|
|[31.4252268808548...|
|[31.4459724827577...|
|[31.4474464941278...|
|[31.5741380228732...|
|[31.6610498227460...|
|[31.7242025238451...|
|[31.8093003166791...|
|[31.8186165667690...|
|[31.9563005605233...|
|[32.0123007682454...|
|[32.0444861274404...|
|[32.0478009788678...|
|[32.0542618511847...|
|[32.0637746203136...|
|[32.0789475795693...|
+--------------------+
only showing top 20 rows



In [32]:
predictions = lr_model.transform(unlabeled_data)

In [33]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.5743636841713...| 441.7978272499961|
|[30.9716756438877...| 487.5720330546733|
|[31.2681042107507...| 426.6629184996227|
|[31.3584771924370...| 490.9782913607612|
|[31.3895854806643...| 409.3934201360414|
|[31.4252268808548...| 534.8366637375602|
|[31.4459724827577...| 482.1014598505967|
|[31.4474464941278...| 426.1981773161392|
|[31.5741380228732...| 558.7490309786192|
|[31.6610498227460...| 416.7946757375937|
|[31.7242025238451...|509.25949588366416|
|[31.8093003166791...| 547.8141563834181|
|[31.8186165667690...| 447.9869173101331|
|[31.9563005605233...| 564.9969840519395|
|[32.0123007682454...|489.25476759136745|
|[32.0444861274404...| 446.5752829823548|
|[32.0478009788678...| 508.2033667997025|
|[32.0542618511847...| 557.2380248428524|
|[32.0637746203136...| 389.5515122599104|
|[32.0789475795693...|  351.821411984002|
+--------------------+------------

^ For the first customer, we expect them to pay $441, etc.